---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [193]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [478]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

states_map = (pd.DataFrame
              .from_dict(states, orient='index')
              .reset_index()
              .rename(columns={'index': 'state_short', 0: 'State'})
             )

states_map.head()

,state_short,State
0,OH,Ohio
1,KY,Kentucky
2,AS,American Samoa
3,NV,Nevada
4,WY,Wyoming


In [195]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    text_file = open('university_towns.txt')
    lines = text_file.read().split('\n')
    towns = []
    for line in lines[:-1]:
        if re.findall('\[edit\]', line): 
            state = line
        else: 
            towns.append({'State': state, 'RegionName': line})      
    towns_df = pd.DataFrame.from_dict(towns)
    towns_df = (towns_df
                .replace(regex={'\[edit\]': ''})
                .replace(regex={'\s\(.*': ''})
                .replace(regex={'\n': ''})
               )
    return towns_df[['State', 'RegionName']]
  

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [419]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = (pd.read_excel('gdplev.xls', skiprows=5, usecols=[4,6])
           .rename(columns={'GDP in billions of chained 2009 dollars.1': 'GDP', 'Unnamed: 4':'Quarter'})
           .set_index('Quarter')
           .assign(Recession_Start = False)
           .assign(Recession_End = False)
           .loc['2000q1':]
          )
    for i in range(0, len(gdp)):
        if ((gdp['GDP'].iloc[i] - gdp['GDP'].iloc[i-1]) < 0) & ((gdp['GDP'].iloc[i-1] - gdp['GDP'].iloc[i-2]) <0):
            gdp['Recession_Start'].iloc[i-1] = True
            break
    return gdp[gdp['Recession_Start'] == True].index[0]

get_recession_start()

'2008q3'

In [424]:
def get_recession_end():
    gdp = (pd.read_excel('gdplev.xls', skiprows=5, usecols=[4,6])
           .rename(columns={'GDP in billions of chained 2009 dollars.1': 'GDP', 'Unnamed: 4':'Quarter'})
           .set_index('Quarter')
           .assign(Recession_Start = False)
           .assign(Recession_End = False)
           .loc[get_recession_start():]
          )
    for i in range(0, len(gdp)):
        if ((gdp['GDP'].iloc[i] - gdp['GDP'].iloc[i-1]) < 0) & ((gdp['GDP'].iloc[i-1] - gdp['GDP'].iloc[i-2]) <0):
            gdp['Recession_Start'].iloc[i-1] = True
            break
    for j in range(0, len(gdp)):
        if ((gdp['GDP'].iloc[j] - gdp['GDP'].iloc[j-1] > 0)) & (gdp['GDP'].iloc[j-1] - gdp['GDP'].iloc[j-2] > 0):
            gdp['Recession_End'].iloc[j] = True
            break
    return gdp[gdp['Recession_End'] == True].index[0]
        
get_recession_end()

'2009q4'

In [425]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = (pd.read_excel('gdplev.xls', skiprows=5, usecols=[4,6])
           .rename(columns={'GDP in billions of chained 2009 dollars.1': 'GDP', 'Unnamed: 4':'Quarter'})
           .set_index('Quarter')
           .assign(Recession_Start = False)
           .assign(Recession_End = False)
           .loc[get_recession_start():get_recession_end()]
          )
    return gdp['GDP'].idxmin()

get_recession_bottom()

'2009q2'

In [484]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = (pd.read_csv('City_Zhvi_AllHomes.csv')
               .rename(columns={'State': 'state_short'})
              )
    housing = (pd.merge(housing, states_map, left_on="state_short", right_on="state_short")
              .set_index(['State', 'RegionName'])
              )
    housing = (housing
               .loc[:,'2000-01':]
               .groupby(pd.PeriodIndex(housing.loc[:,'2000-01':].columns, freq='Q'), axis=1)
               .mean()
               .rename(columns=lambda c: str(c).lower())
             )       
    return housing

convert_housing_data_to_quarters()

2000q1         2000q2  \
State         RegionName                                                
New York      New York                             NaN            NaN   
              Rochester                   53566.666667   53233.333333   
              Yonkers                    222233.333333  230900.000000   
              Syracuse                    56800.000000   57866.666667   
              Town of Greece              95100.000000   95500.000000   
              Albany                      91566.666667   92100.000000   
              Town of Poughkeepsie       131966.666667  135433.333333   
              New Rochelle               362766.666667  370866.666667   
              Town of Colonie            110600.000000  112400.000000   
              Mount Vernon               239333.333333  247300.000000   
              Utica                       50700.000000   51233.333333   
              Valley Stream              223166.666667  225166.666667   
              Schenectady                 72266.666667   73100.000000   
              White Plains               284333.333333  297833.333333   
              Town of Hempstead          170166.666667  172266.666667   
              Town of Binghamton          81633.333333   82500.000000   
              Town of Newburgh           116000.000000  118400.000000   
              Troy                        74733.333333   76533.333333   
              Niagara Falls               50300.000000   50133.333333   
              Irondequoit                 83400.000000   83500.000000   
              Binghamton                  51766.666667   52000.000000   
              Massapequa                 248833.333333  251233.333333   
              Town of Elmira              51200.000000   51600.000000   
              Henrietta                   94566.666667   94400.000000   
              Town of Webster            126433.333333  128600.000000   
              Patchogue                  134400.000000  135200.000000   
              Clifton Park               140633.333333  142500.000000   
              Freeport                   183833.333333  186433.333333   
              Lindenhurst                168733.333333  169100.000000   
              Levittown                  189966.666667  193366.666667   
              Elmont                     198666.666667  199400.000000   
              West Babylon               168066.666667  169333.333333   
              Hicksville                 215133.333333  217733.333333   
              Perinton                   139433.333333  140766.666667   
              Great Neck                 570900.000000  576700.000000   
              East Meadow                219300.000000  223166.666667   
              Penfield                   134200.000000  135966.666667   
              Ronkonkoma                 160433.333333  160900.000000   
              Saratoga Springs           133633.333333  137833.333333   
              Brighton                   120900.000000  120933.333333   
              Rome                        53800.000000   54466.666667   
              Merrick                    269633.333333  275866.666667   
              Town of Lockport            74866.666667   75533.333333   
              Bellmore                   242566.666667  248333.333333   
              Farmingdale                215566.666667  218733.333333   
              North Tonawanda             73766.666667   73833.333333   
              Jamestown                   45800.000000   45900.000000   
              Town of Watertown           52133.333333   52566.666667   
              Baldwin                    215533.333333  218300.000000   
              Town of Orchard Park                 NaN            NaN   
              New City                   275433.333333  290833.333333   
              Port Chester               356850.000000  366733.333333   
              Chili                      100466.666667  100900.000000   
              Ossining           

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"